In [35]:
#!pip install numpy
!pip install pandas
!pip install tqdm


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.0-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [38]:
#import numpy as np
import itertools
import csv
import pandas as pd
import time

In [39]:
def read_and_generate_transactionslist():
    tr_file = "T10I4D100K.dat"
    
    #Read transactions file and convert the format to a list of transaction-lists
    # the transaction values are integers
    with open(tr_file, 'r') as file:
        data_set = [list(map(int, line.split())) for line in file]
    return data_set

In [40]:
def construct(k, Lk_minus, data_set):
    Ck = set()
    if k == 1:
        min_val = min(value for transaction in data_set for value in transaction)
        max_val = max(value for transaction in data_set for value in transaction)
        unique_values = [[uv] for uv in range(min_val,max_val+1)]
        Ck = unique_values
    else:
        """
        for i in range(len(Lk_minus)):
            itemset_i = Lk_minus[i]
            if i+1 < len(Lk_minus):
                #remaining_sublist = Lk_minus[i+1:]
                for itemset_j in Lk_minus[i+1:]:
                    for value in itemset_j:
                        if value not in itemset_i:
                            candidate_i = list(itemset_i.copy())
                            candidate_i.append(value)
                            Ck.add(tuple(candidate_i))
        """
        #picks all possible combinations of two itemsets; itemset_i, itemset_j
        for itemset_i, itemset_j in itertools.combinations(Lk_minus, 2):
            union_i = set(itemset_i).union(itemset_j)
            # generate all possible combinations of length k from the union of the old itemsets
            for candidate_i in itertools.combinations(union_i, k):  
                #add to a set in order to prevent duplicate candidates
                Ck.add(tuple(candidate_i))  
        Ck = list(Ck)
                
    return Ck

In [41]:
def calculate_support(itemset_i, data_set):
    itemset_i_set = set(itemset_i)  
    occurences = 0
    for transaction in data_set:
        transaction_set = set(transaction)  
        if itemset_i_set.issubset(transaction_set): 
            occurences += 1
    support = occurences / len(data_set)
    return support

In [42]:
def filter(s, Ck, data_set):
    """
    Lk = []
    for itemset_i in Ck:
        occurences = 0
        for transaction in data_set:
            occurences += np.sum(np.all(np.isin(np.array(list(itemset_i)), np.array(transaction))))
        support = occurences/len(data_set)
        if support>=s:
            Lk.append(list(itemset_i))
    """
    Lk = []
    for itemset_i in Ck:
        support = calculate_support(itemset_i, data_set)
        if support >= s:
            Lk.append(list(itemset_i))
    return Lk

In [66]:
def apriori(k, data_set):
    #support s, threshold of minimum transactions containing the itemset:
    s = 0.01
    #max tr length:
    #tl = len(max(data_set, key=len))
    # use k = 3
    Lk_minus = None
    for k in range(1,k+1):
        start_constr_k = time.time()
        Ck = construct(k, Lk_minus, data_set)
        end_constr_k = time.time()
        print(f"elapsed time to construct C{k}: {end_constr_k-start_constr_k} seconds")
        start_filter_k = time.time()
        Lk = filter(s, Ck, data_set)
        end_filter_k = time.time()
        print(f"elapsed time to filter C{k} into L{k}: {end_filter_k-start_filter_k} seconds")
        Lk_minus = Lk
    freq_itemsets = Lk
    return freq_itemsets

In [67]:
def generate_association_rules(k, freq_itemsets, data_set):
    association_rules = "supported_association_rules.csv"
    # confidence, threshold for association rules
    c = 0.50
    #dont think this is needed, since already calculated:
    #s = 0.01
    # Create and write the headers to the CSV file
    with open(association_rules, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['X', 'Y', 'Support', 'Confidence'])
        
        # go through each itemset in freq_itemsets:
        for freq_set in freq_itemsets:
            # split them in each possible combination X, Y, such that S=X⋃Y :
            #generates all possible combinations of length k-1 of the current frequent itemset
            for X in itertools.combinations(freq_set, k-1):
                Y = set(freq_set) - set(X)
        
                # calculate the support of the association rule X->Y, i.e. the number of transactions that contain X⋃Y=S, I use fraction
                #S = freq_set
                support_S = calculate_support(freq_set, data_set)
            
                # calculate the confidence of the association rule X->Y, i.e. the fraction of transactions containing X⋃Y 
                # in all transactions that contain X. support(S)/support(X)
                support_X = calculate_support(X, data_set)
                confidence_i = support_S/support_X
                # if the association rule passes the thresholds of c and s, add the sheet of supported association rules:
                if confidence_i>= c:
                    writer.writerow([X, Y, support_S, confidence_i])
    
    df = pd.read_csv(association_rules)
    print(df)

In [ ]:
#read transactions file and generate it to a list of transactions-lists:
data_set = read_and_generate_transactionslist()
#print(len(data_set)) #100,000

k = 3
start_apriori = time.time()
# generate the list of frequent itemsets (a list of lists):
freq_itemsets = apriori(k, data_set)
end_apriori = time.time()
print(f"total time to obtain the frequent itemsets using the apriori algorithm: {end_apriori-start_apriori} seconds")
#print(freq_itemsets)

# evaluate association rules that are supported by the support and confidense criterias
# and visualize their representation in a csv-format:
start_association_rules = time.time()
generate_association_rules(k, freq_itemsets, data_set)
end_association_rules = time.time()
print(f"total time to evaluate association rules: {end_association_rules-start_association_rules} seconds")

elapsed time to construct C1: 0.1537792682647705 seconds
elapsed time to filter C1 into L1: 89.65252733230591 seconds
elapsed time to construct C2: 0.08660888671875 seconds
